### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import cv2

from IPython.display import clear_output
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

### Load Model

In [2]:
model = tf.keras.models.load_model('./video_anomaly_classifier_20-EPOCHS_SGD/')

In [3]:
NUM_CLASSES = 14
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting',"Normal",'RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism']

### Predict Using Videos

In [4]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
BATCH_SIZE = 1
SEED = 12

In [10]:
video_path = './data/video/accidents/accident_1.mp4'
video = cv2.VideoCapture(video_path)

while True:
    
    ret, frame = video.read()
    
    if frame is None: #if reaches the end of the video and there is no more image
        print('Completed!')
        break
    
    # preprocess frame
    frame_input = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_input = cv2.resize(frame_input, (IMAGE_HEIGHT,IMAGE_WIDTH), interpolation=cv2.INTER_AREA)
    frame_input = tf.keras.applications.densenet.preprocess_input(frame_input)
    frame_input = frame_input.astype('float') / 255.0 #normalize by scaling values to between 0 and 1
    frame_input = np.expand_dims(frame_input, axis=0)
    
    prediction = model.predict(frame_input)
    predicted_class = CLASS_LABELS[np.argmax(prediction)]
    prediction_score = prediction[0][np.argmax(prediction)]

    if prediction_score > 0.8:
        cv2.putText(frame, "{} {:.2f}".format(predicted_class, prediction_score), (0,30), 0, 1, (255,255,255), 2)
    
    cv2.imshow('video', frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'): #press Q to quit video
        break

video.release() #end video capturing
cv2.destroyAllWindows()
cv2.waitKey(1)

-1